In [7]:
import pandas as pd
import numpy as np

In [2]:
vdjdbfile = pd.read_csv('vdjdb_data.tsv', sep='\t')

In [3]:
# • Species: Human
# • Chain type: TRB (CDR3-beta), TRA (CDR3-alpha, paired only)
chainTypes = ['TRA', 'TRB']
# • HLA type: HLA-A*02:01, All
HLATypes = ['HLA-A*02:01', None]
# • MHC class: I, II
MHCclasses = ['MHCI', 'MHCII']
# • Score: 0, 1, 2, 3, 1-3
possibleScores = [0, 1, 2, 3, None]
vdjdbfile

,complex.id,Gene,CDR3,V,J,Species,MHC A,MHC B,MHC class,Epitope,Epitope gene,Epitope species,Reference,Method,Meta,CDR3fix,Score
0,0,TRA,CAVTDDKIIF,TRAV12-2*01,TRAJ30*01,HomoSapiens,HLA-A*02,B2M,MHCI,LLWNGPMAV,NS4B,YFV,PMID:28975614,"{""frequency"": ""6/56"", ""identification"": ""tetra...","{""cell.subset"": ""CD8"", ""clone.id"": """", ""donor....","{""cdr3"": ""CAVTDDKIIF"", ""cdr3_old"": ""CAVTDDKIIF...",1
1,0,TRA,CAVDSGGYQKVTF,TRAV12-2*01,TRAJ13*01,HomoSapiens,HLA-A*02,B2M,MHCI,LLWNGPMAV,NS4B,YFV,PMID:28975614,"{""frequency"": ""1/56"", ""identification"": ""tetra...","{""cell.subset"": ""CD8"", ""clone.id"": """", ""donor....","{""cdr3"": ""CAVDSGGYQKVTF"", ""cdr3_old"": ""CAVDSGG...",0
2,0,TRA,CAGGDDKIIF,TRAV12-2*01,TRAJ30*01,HomoSapiens,HLA-A*02,B2M,MHCI,LLWNGPMAV,NS4B,YFV,PMID:28975614,"{""frequency"": ""1/56"", ""identification"": ""tetra...","{""cell.subset"": ""CD8"", ""clone.id"": """", ""donor....","{""cdr3"": ""CAGGDDKIIF"", ""cdr3_old"": ""CAGGDDKIIF...",0
3,0,TRA,CAVKDARLMF,TRAV12-2*01,TRAJ31*01,HomoSapiens,HLA-A*02,B2M,MHCI,LLWNGPMAV,NS4B,YFV,PMID:28975614,"{""frequency"": ""3/56"", ""identification"": ""tetra...","{""cell.subset"": ""CD8"", ""clone.id"": """", ""donor....","{""cdr3"": ""CAVKDARLMF"", ""cdr3_old"": ""CAVKDARLMF...",1
4,0,TRA,CAVGSDKIIF,TRAV12-2*01,TRAJ30*01,HomoSapiens,HLA-A*02,B2M,MHCI,LLWNGPMAV,NS4B,YFV,PMID:28975614,"{""frequency"": ""4/56"", ""identification"": ""tetra...","{""cell.subset"": ""CD8"", ""clone.id"": """", ""donor....","{""cdr3"": ""CAVGSDKIIF"", ""cdr3_old"": ""CAVGSDKIIF...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67684,24463,TRA,CWSPFGNEKLTF,TRAV8-2*01,TRAJ48*01,HomoSapiens,HLA-A*11:01,B2M,MHCI,GTSGSPIVNR,NS3,DENV1,PMID:28945243,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD3+CD8+CD14-CD19-"", ""clone.i...","{""cdr3"": ""CWSPFGNEKLTF"", ""cdr3_old"": ""CWSPFGNE...",2
67685,24464,TRA,CWSPFGNEKLTF,TRAV8-2*01,TRAJ48*01,HomoSapiens,HLA-A*11:01,B2M,MHCI,GTSGSPIINR,NS3,DENV3/4,PMID:28945243,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD3+CD8+CD14-CD19-"", ""clone.i...","{""cdr3"": ""CWSPFGNEKLTF"", ""cdr3_old"": ""CWSPFGNE...",2
67686,24476,TRA,CAVTPDGQKLLF,TRAV1-1*01,NaN,HomoSapiens,HLA-A*02:01,B2M,MHCI,NLVPMVATV,pp65,CMV,https://github.com/antigenomics/vdjdb-db/issue...,"{""frequency"": ""95%"", ""identification"": ""pelime...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAVTPDGQKLLF"", ""cdr3_old"": ""CAVTPDGQ...",1
67687,24494,TRA,SSGNQFYF,TRAV21*01,TRAJ49*01,HomoSapiens,HLA-A*02,B2M,MHCI,NLVPMVATV,pp65,CMV,PMID:28623251,"{""frequency"": ""67.5%"", ""identification"": ""tetr...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""SSGNQFYF"", ""cdr3_old"": ""SSGNQFYF"", ""...",3


In [14]:
# Determine counts for VDJDB
counts = {}
for chain in chainTypes:
    for mhc in MHCclasses:
        for hla in HLATypes:
            if hla == None:
                for score in possibleScores:
                    label = (('Species','Human'), ('Chain', chain), ('MHC', mhc), ('HLA', hla), ('Score', score))
                    if score == None:
                        temp = vdjdbfile[(vdjdbfile['Species'] == 'HomoSapiens') & \
                                         (vdjdbfile['Gene'] == chain) & \
                                         (vdjdbfile['Score'] > 0) & \
                                         (vdjdbfile['MHC class'] == mhc)]
                                        
                        count = 'Pairs:' + str(temp.shape[0]) + \
                                ', Unique CDR3s: ' + str(len(temp['CDR3'].unique())) + \
                                ', Unique epitopes: ' + str(len(temp['Epitope'].unique()))
                        counts[label] = count
                    else:
                        temp = vdjdbfile[(vdjdbfile['Species'] == 'HomoSapiens') & \
                                         (vdjdbfile['Gene'] == chain) & \
                                         (vdjdbfile['MHC class'] == mhc) & \
                                         (vdjdbfile['Score'] == score)]
                        count = 'Pairs:' + str(temp.shape[0]) + \
                                ', Unique CDR3s: ' + str(len(temp['CDR3'].unique())) + \
                                ', Unique epitopes: ' + str(len(temp['Epitope'].unique()))
                        counts[label] = count
                
            else:
                for score in possibleScores:
                    label = (('Species','Human'), ('Chain', chain), ('MHC', mhc), ('HLA', hla), ('Score', score))
                    if score == None:
                        temp = vdjdbfile[(vdjdbfile['Species'] == 'HomoSapiens') & \
                                          (vdjdbfile['Gene'] == chain) & \
                                          (vdjdbfile['MHC A'] == hla) & \
                                          (vdjdbfile['Score'] > 0) & \
                                          (vdjdbfile['MHC class'] == mhc)]
                        count = 'Pairs:' + str(temp.shape[0]) + \
                                ', Unique CDR3s: ' + str(len(temp['CDR3'].unique())) + \
                                ', Unique epitopes: ' + str(len(temp['Epitope'].unique()))
                        counts[label] = count
                    else:
                        temp = vdjdbfile[(vdjdbfile['Species'] == 'HomoSapiens') & \
                                         (vdjdbfile['Gene'] == chain) & \
                                         (vdjdbfile['MHC A'] == hla) & \
                                         (vdjdbfile['MHC class'] == mhc) & \
                                         (vdjdbfile['Score'] == score)]
                        count = 'Pairs:' + str(temp.shape[0]) + \
                                ', Unique CDR3s: ' + str(len(temp['CDR3'].unique())) + \
                                ', Unique epitopes: ' + str(len(temp['Epitope'].unique()))
                        counts[label] = count

In [15]:
for count in counts:
    print(count)
    print(counts[count])
    print('\n')

(('Species', 'Human'), ('Chain', 'TRA'), ('MHC', 'MHCI'), ('HLA', 'HLA-A*02:01'), ('Score', 0))
Pairs:3800, Unique CDR3s: 2484, Unique epitopes: 29


(('Species', 'Human'), ('Chain', 'TRA'), ('MHC', 'MHCI'), ('HLA', 'HLA-A*02:01'), ('Score', 1))
Pairs:445, Unique CDR3s: 264, Unique epitopes: 6


(('Species', 'Human'), ('Chain', 'TRA'), ('MHC', 'MHCI'), ('HLA', 'HLA-A*02:01'), ('Score', 2))
Pairs:13, Unique CDR3s: 7, Unique epitopes: 1


(('Species', 'Human'), ('Chain', 'TRA'), ('MHC', 'MHCI'), ('HLA', 'HLA-A*02:01'), ('Score', 3))
Pairs:5, Unique CDR3s: 1, Unique epitopes: 1


(('Species', 'Human'), ('Chain', 'TRA'), ('MHC', 'MHCI'), ('HLA', 'HLA-A*02:01'), ('Score', None))
Pairs:463, Unique CDR3s: 268, Unique epitopes: 6


(('Species', 'Human'), ('Chain', 'TRA'), ('MHC', 'MHCI'), ('HLA', None), ('Score', 0))
Pairs:26365, Unique CDR3s: 19376, Unique epitopes: 56


(('Species', 'Human'), ('Chain', 'TRA'), ('MHC', 'MHCI'), ('HLA', None), ('Score', 1))
Pairs:736, Unique CDR3s: 420, Unique

In [26]:
cleanedData = vdjdbfile[['Epitope', 'CDR3', 'Score']]
cleanedData.loc[cleanedData['Score'] > 0, 'Score'] = 1
cleanedData.to_csv('VDJDB_CleanedData.csv', index=False, header=False, sep='\t')